# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from sklearn.svm import SVC
from keras.utils import to_categorical
import warnings
import sklearn.exceptions

In [ ]:
warnings.filterwarnings('ignore', category=sklearn.exceptions.UndefinedMetricWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

# Load all Data Sets

In [ ]:
# Main X and Y data
x_train = pd.read_csv('Data/x_train_gr_smpl.csv')
x_test = pd.read_csv('Data/x_test_gr_smpl.csv')

y_train = pd.read_csv('Data/y_train_smpl.csv')
y_test = pd.read_csv('Data/y_test_smpl.csv')


# One-Vs-Rest Y data
# Training data
y_train_0 = pd.read_csv('Data/y_train_smpl_0.csv')
y_train_1 = pd.read_csv('Data/y_train_smpl_1.csv')
y_train_2 = pd.read_csv('Data/y_train_smpl_2.csv')
y_train_3 = pd.read_csv('Data/y_train_smpl_3.csv')
y_train_4 = pd.read_csv('Data/y_train_smpl_4.csv')
y_train_5 = pd.read_csv('Data/y_train_smpl_5.csv')
y_train_6 = pd.read_csv('Data/y_train_smpl_6.csv')
y_train_7 = pd.read_csv('Data/y_train_smpl_7.csv')
y_train_8 = pd.read_csv('Data/y_train_smpl_8.csv')
y_train_9 = pd.read_csv('Data/y_train_smpl_9.csv')

# Testing Data
y_test_0 = pd.read_csv('Data/y_test_smpl_0.csv')
y_test_1 = pd.read_csv('Data/y_test_smpl_1.csv')
y_test_2 = pd.read_csv('Data/y_test_smpl_2.csv')
y_test_3 = pd.read_csv('Data/y_test_smpl_3.csv')
y_test_4 = pd.read_csv('Data/y_test_smpl_4.csv')
y_test_5 = pd.read_csv('Data/y_test_smpl_5.csv')
y_test_6 = pd.read_csv('Data/y_test_smpl_6.csv')
y_test_7 = pd.read_csv('Data/y_test_smpl_7.csv')
y_test_8 = pd.read_csv('Data/y_test_smpl_8.csv')
y_test_9 = pd.read_csv('Data/y_test_smpl_9.csv')

# Create Linear Classifier Function

In [ ]:
def linear_classifier(x_train, y_train, x_test, y_test):
    
    y_train = np.array(y_train).flatten()
    y_test = y_test.values.flatten()
    
    model = SVC()
    model.fit(x_train, y_train)
    
    score = model.score(x_test, y_test)
    
    return print('Score of Linear Classifier:', score)

# Create Neural Network Function, Analyze and Visualize Results

In [ ]:
def neural_network(x_train, y_train, x_test, y_test):
    
    x_train = np.array(x_train)
    x_train = x_train.reshape(len(x_train), 48, 48)
    x_train = x_train/255

    x_test = np.array(x_test)
    x_test = x_test.reshape(len(x_test), 48, 48)
    x_test = x_test/255

    # Neural Network Model
    model = Sequential()
    model.add(Flatten(input_shape = [48,48]))
    
    for i in range(2):
        model.add(Dense(units=100, activation='relu'))
    
    model.add(Dense(units=2, activation='softmax'))
    
    
    opt = SGD(learning_rate=0.01) 
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    model.fit(x_train, y_train, epochs=5)
    y_pred = np.argmax(model.predict(x_test, verbose=0), axis=-1)

    print('---------------------------------------------------------------------------------------------------------------')
    print('\n')
    print('Results')
    # Analyzing the results
    # Model Accuracy
    print('\n')
    scores = model.evaluate(x_test, y_test)
    print(f'{model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    print('\n')
    
    # Confusion Matrix
    cm = metrics.confusion_matrix(y_test, y_pred)
    print('Confusion Matrix:')
    print(cm)
    print('\n')
    
    # True Positive
    TP = np.diag(cm)
    # False Positive
    FP = cm.sum(axis=0) - TP
    # False Negative
    FN = cm.sum(axis=1) - TP
    # True Negative
    TN = cm.sum() - (FP+FN+TP)


    #True Positive Rate
    TPR = TP/(TP+FN)
    TPR_mean = np.round(TPR.mean(), 2)
    #False Positive Rate
    FPR = FP/(FP+TN)
    FPR_mean = np.round(FPR.mean(), 2)
    #Precision
    Precision = TP/(TP+FP)
    Precision_mean = np.round(Precision.mean(), 2)
    #Recall
    Recall = TP/(TP+FN)
    Recall_mean = np.round(Recall.mean(), 2)
    #F1 Measure
    F1 = 2 * (Precision * Recall) / (Precision + Recall)
    F1_mean = np.round(F1.mean(), 2)
    
    roc_auc = metrics.roc_auc_score(y_test, y_pred)
    
    np.set_printoptions(precision=2)
    print('---------------------------------------------------------------------------------------------------------------')
    print('\n')
    print('TP rate: ', TPR)
    print('Mean TP rate: ', TPR_mean)
    print('FP rate: ', FPR)
    print('Mean FP rate: ', FPR_mean)
    print('Precision: ', Precision)
    print('Mean Precision: ', Precision_mean)
    print('Recall: ', Recall)
    print('Mean Recall: ', Recall_mean)
    print('F Measure: ', F1)
    print('Mean F Measure: ', F1_mean)
    print('ROC Area: ', roc_auc)
    print('\n')
    
    
    print('Classification Report:')
    print(metrics.classification_report(y_test, y_pred))
    
    # Visualizing the results
    # Heatmap
    print('---------------------------------------------------------------------------------------------------------------')
    print('\n')
    print('\n')
    print('Visualization of Results')
    plt.figure(figsize=(11,7))
    sns.heatmap(cm, annot=True, fmt=' .0f', square=True, cmap='BuPu')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Heatmap of Confusion Matrix')
    
    plt.figure(figsize=(8,4))
    plt.scatter(Precision, Recall, c='purple')
    plt.xlabel('Precsion')
    plt.ylabel('Recall')
    plt.title('Precision vs Recall Scatter Chart')
    plt.show()
    print('---------------------------------------------------------------------------------------------------------------')

# Create Convolutional Neural Network, Analyze and Visualize Results

In [ ]:
def conv_neural_network(x_train, y_train, x_test, y_test):    
    
    x_train = np.array(x_train)
    x_train = x_train.reshape((len(x_train), 48, 48, 1))
    x_train = x_train/255

    x_test = np.array(x_test)
    x_test = x_test.reshape((len(x_test), 48, 48, 1))
    x_test = x_test/255

    
    y_train = to_categorical(y_train)
    y_test_ = y_test.copy()
    y_test = to_categorical(y_test)
    
    
    # Neural Network Model
    model = Sequential()

    model.add(Conv2D(32, (3,3), input_shape=(48,48,1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())
    model.add(Dense(100))

    model.add(Dense(2))
    model.add(Activation('softmax'))

    opt = SGD(learning_rate=0.01, momentum=0.9)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=5, batch_size=48)
    
    y_pred = np.argmax(model.predict(x_test, verbose=0), axis=-1)
    
    print('---------------------------------------------------------------------------------------------------------------')
    print('\n')
    print('Results')
    # Analyzing the results
    # Model Accuracy
    print('\n')
    scores = model.evaluate(x_test, y_test)
    print(f'{model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    print('\n')
    
    # Confusion Matrix
    y_test = y_test_.copy()
    cm = metrics.confusion_matrix(y_test, y_pred)
    print('Confusion Matrix:')
    print(cm)
    print('\n')
    
    # True Positive
    TP = np.diag(cm)
    # False Positive
    FP = cm.sum(axis=0) - TP
    # False Negative
    FN = cm.sum(axis=1) - TP
    # True Negative
    TN = cm.sum() - (FP+FN+TP)


    #True Positive Rate
    TPR = TP/(TP+FN)
    TPR_mean = np.round(TPR.mean(), 2)
    #False Positive Rate
    FPR = FP/(FP+TN)
    FPR_mean = np.round(FPR.mean(), 2)
    #Precision
    Precision = TP/(TP+FP)
    Precision_mean = np.round(Precision.mean(), 2)
    #Recall
    Recall = TP/(TP+FN)
    Recall_mean = np.round(Recall.mean(), 2)
    #F1 Measure
    F1 = 2 * (Precision * Recall) / (Precision + Recall)
    F1_mean = np.round(F1.mean(), 2)
    
    roc_auc = metrics.roc_auc_score(y_test, y_pred)
    
    np.set_printoptions(precision=2)
    print('---------------------------------------------------------------------------------------------------------------')
    print('\n')
    print('TP rate: ', TPR)
    print('Mean TP rate: ', TPR_mean)
    print('FP rate: ', FPR)
    print('Mean FP rate: ', FPR_mean)
    print('Precision: ', Precision)
    print('Mean Precision: ', Precision_mean)
    print('Recall: ', Recall)
    print('Mean Recall: ', Recall_mean)
    print('F Measure: ', F1)
    print('Mean F Measure: ', F1_mean)
    print('ROC Area: ', roc_auc)
    print('\n')
    
    print('Classification Report:')
    print(metrics.classification_report(y_test, y_pred))
    
    # Visualizing the results
    # Heatmap
    print('---------------------------------------------------------------------------------------------------------------')
    print('\n')
    print('\n')
    print('Visualization of Results')
    plt.figure(figsize=(11,7))
    sns.heatmap(cm, annot=True, fmt=' .0f', square=True, cmap='Set1')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Heatmap of Confusion Matrix')
    
    plt.figure(figsize=(8,4))
    plt.scatter(Precision, Recall, c='r')
    plt.xlabel('Precsion')
    plt.ylabel('Recall')
    plt.title('Precision vs Recall Scatter Chart')
    plt.show()
    print('---------------------------------------------------------------------------------------------------------------')

# One-vs-Rest Classification

# Y_train_0 and Y_test_0

In [ ]:
linear_classifier(x_train, y_train_0, x_test, y_test_0)

In [ ]:
neural_network(x_train, y_train_0, x_test, y_test_0)

In [ ]:
conv_neural_network(x_train, y_train_1, x_test, y_test_1 )

# Y_train_1 and Y_test_1

In [ ]:
linear_classifier(x_train, y_train_1, x_test, y_test_1)

In [ ]:
neural_network(x_train, y_train_1, x_test, y_test_1)

In [ ]:
conv_neural_network(x_train, y_train_1, x_test, y_test_1 )

# Y_train_2 and Y_test_2

In [ ]:
linear_classifier(x_train, y_train_2, x_test, y_test_2)

In [ ]:
neural_network(x_train, y_train_2, x_test, y_test_2)

In [ ]:
conv_neural_network(x_train, y_train_2, x_test, y_test_2 )

# Y_train_3 and Y_test_3

In [ ]:
linear_classifier(x_train, y_train_3, x_test, y_test_3)

In [ ]:
neural_network(x_train, y_train_3, x_test, y_test_3)

In [ ]:
conv_neural_network(x_train, y_train_3, x_test, y_test_3 )

# Y_train_4 and Y_test_4

In [ ]:
linear_classifier(x_train, y_train_4, x_test, y_test_4)

In [ ]:
neural_network(x_train, y_train_4, x_test, y_test_4)

In [ ]:
conv_neural_network(x_train, y_train_4, x_test, y_test_4 )

# Y_train_5 and Y_test_5

In [ ]:
linear_classifier(x_train, y_train_5, x_test, y_test_5)

In [ ]:
neural_network(x_train, y_train_5, x_test, y_test_5)

In [ ]:
conv_neural_network(x_train, y_train_5, x_test, y_test_5 )

# Y_train_6 and Y_test_6

In [ ]:
linear_classifier(x_train, y_train_6, x_test, y_test_6)

In [ ]:
neural_network(x_train, y_train_6, x_test, y_test_6)

In [ ]:
conv_neural_network(x_train, y_train_6, x_test, y_test_6 )

# Y_train_7 and 7

In [ ]:
linear_classifier(x_train, y_train_7, x_test, y_test_7)

In [ ]:
neural_network(x_train, y_train_7, x_test, y_test_7)

In [ ]:
conv_neural_network(x_train, y_train_7, x_test, y_test_7 )

# Y_train_8 and Y_test_8

In [ ]:
linear_classifier(x_train, y_train_8, x_test, y_test_8)

In [ ]:
neural_network(x_train, y_train_8, x_test, y_test_8)

In [ ]:
conv_neural_network(x_train, y_train_8, x_test, y_test_8 )

# Y_train_9 and Y_test_9

In [ ]:
linear_classifier(x_train, y_train_9, x_test, y_test_9)

In [ ]:
neural_network(x_train, y_train_9, x_test, y_test_9)

In [ ]:
conv_neural_network(x_train, y_train_9, x_test, y_test_9 )